In [5]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install langchain_text_splitters 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=350,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

with open('cricket.txt', mode = 'r', encoding = 'utf-8') as file:
    text = file.read()

text_chunks = text_splitter.create_documents([text])

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import BertTokenizer, BertModel
import torch
from langchain.vectorstores import Chroma
from langchain.schema import Document
class bert_model_embeddings:
    def __init__(self, modelname = 'bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(modelname)
        self.model = BertModel.from_pretrained(modelname)

# Step 2: Define a Function to Get Embeddings
    def embed_documents(self,texts):
        """
        Generate embeddings for a list of texts using BERT.
        """
        
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling
            #print(embeddings)
        return embeddings.numpy().tolist()
    
    def embed_query(self,query):
        inputs = self.tokenizer(query, padding =True, truncation = True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        query_embedding = outputs.last_hidden_state.mean(dim=1)
        return query_embedding.numpy().tolist()
# Step 3: Prepare Documents
texts = [doc.page_content for doc in text_chunks]
print(texts)

model = "bert-base-uncased"
myembedmodel = bert_model_embeddings(model)

from langchain.vectorstores import Chroma
# Step 4: Create Chroma Vector Store
vector_store = Chroma.from_texts(
    texts = texts,
    #documents=text_chunks,
    collection_name="wiki_collection",
    embedding=myembedmodel,  # Correctly use the 'embedding' parameter
    #metadatas=[doc.metadata for doc in text_chunks],
   
    persist_directory="c_aricle"
)

# # Step 5: Query the Vector Store
query = "What is cricket?"
query_embedding = myembedmodel.embed_query(query)

#query_embedding = bert_model_embeddings([query])[0]

# Step 6: Perform Similarity Search

results = vector_store.similarity_search(query, k=3)
results

['First-class cricket\nOne Day International\nLimited overs (domestic)\nTwenty20 International\nTwenty20 (domestic)\nOther forms', 'Cricket is a bat-and-ball game played between two teams of eleven players on a field, at the centre of which is a 22-yard (20-metre; 66-foot) pitch with a wicket at each end, each comprising two bails (small sticks) balanced on three stumps. Two players from the batting team, the striker and nonstriker, stand in front of either wicket holding', "wicket holding bats, while one player from the fielding team, the bowler, bowls the ball toward the striker's wicket from the opposite end of the pitch. The striker's goal is to hit the bowled ball with the bat and then switch places with the nonstriker, with the batting team scoring one run for each of these exchanges. Runs are also scored when", 'also scored when the ball reaches the boundary of the field or when the ball is bowled illegally.', 'The fielding team tries to prevent runs from being scored by dismiss

ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[[0.18503010272979736, 0.07308343052864075, -0.4451271593570709, -0.2142052799463272, -0.04939587786793709, -0.37588706612586975, 0.4569190442562103, 0.5618745684623718, -0.21250873804092407, -0.38410162925720215, 0.05026206001639366, -0.40699586272239685, -0.32636746764183044, 0.14562766253948212, -0.21568946540355682, 0.3793604373931885, 0.29553332924842834, 0.17561596632003784, 0.0015740742674097419, 0.5377097725868225, -0.2291916161775589, 0.06992030143737793, -0.4394585192203522, 0.14699159562587738, 0.4094964265823364, 0.26670315861701965, -0.3145260512828827, -0.042983222752809525, 0.19208955764770508, -0.3758317530155182, -0.06859175860881805, 0.15037687122821808, 0.05964790657162666, -0.03313680365681648, -0.43582868576049805, -0.4538774788379669, -0.2527245581150055, -0.010636702179908752, -0.21208946406841278, 0.02743806503713131, -0.699067533016204

In [9]:
# Install required packages
# %pip install langchain transformers chromadb

from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BertTokenizer, BertModel
import torch
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings

# Step 1: Text Splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=20,
    length_function=len
)

with open('cricket.txt', mode='r', encoding='utf-8') as file:
    text = file.read()

text_chunks = text_splitter.create_documents([text])

# Step 2: Define Custom Embedding Class
class BertModelEmbeddings(Embeddings):
    def __init__(self, modelname='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(modelname)
        self.model = BertModel.from_pretrained(modelname)

    def embed_documents(self, texts):
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.numpy().tolist()

    def embed_query(self, query):
        inputs = self.tokenizer(query, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        query_embedding = outputs.last_hidden_state.mean(dim=1)
        return query_embedding.numpy().tolist()

# Step 3: Initialize Embedding Model
model = "bert-base-uncased"
myembedmodel = BertModelEmbeddings(modelname=model)

# Step 4: Create Chroma Vector Store
texts = [doc.page_content for doc in text_chunks]
metadatas = [doc.metadata for doc in text_chunks]  # Optional: Include metadata if available

vector_store = Chroma.from_texts(
    texts=texts,
    embedding=myembedmodel,
    collection_name="wiki_collection",
    metadatas=metadatas,
    persist_directory="c_article"
)

# Step 5: Query the Vector Store
query = "What is cricket?"
results = vector_store.similarity_search(query, k=3)

# Output results
for result in results:
    print(result)


ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[[0.18503010272979736, 0.07308343052864075, -0.4451271593570709, -0.2142052799463272, -0.04939587786793709, -0.37588706612586975, 0.4569190442562103, 0.5618745684623718, -0.21250873804092407, -0.38410162925720215, 0.05026206001639366, -0.40699586272239685, -0.32636746764183044, 0.14562766253948212, -0.21568946540355682, 0.3793604373931885, 0.29553332924842834, 0.17561596632003784, 0.0015740742674097419, 0.5377097725868225, -0.2291916161775589, 0.06992030143737793, -0.4394585192203522, 0.14699159562587738, 0.4094964265823364, 0.26670315861701965, -0.3145260512828827, -0.042983222752809525, 0.19208955764770508, -0.3758317530155182, -0.06859175860881805, 0.15037687122821808, 0.05964790657162666, -0.03313680365681648, -0.43582868576049805, -0.4538774788379669, -0.2527245581150055, -0.010636702179908752, -0.21208946406841278, 0.02743806503713131, -0.699067533016204